In [3]:
import speech_recognition as sr
import pyttsx3
import os
import subprocess
import csv
import subprocess
from difflib import SequenceMatcher
import heapq
from typing import Dict, List, Tuple

In [4]:
def speak(text):
    engine = pyttsx3.init('espeak')
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[0].id)

    # engine.setProperty()
    engine.say(text)
    engine.runAndWait()

In [33]:
speak("how are you brother i am fine thank you")

In [15]:
def listen():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        recognizer.pause_threshold = 1
        recognizer.adjust_for_ambient_noise(source)
        try:
            audio = recognizer.listen(source, 10, 10)
            query = recognizer.recognize_google(audio, language = "en-in")
            print("User said: ",query)
            return query.lower()
        except sr.UnknownValueError:
            return None
        except sr.RequestError as e:
            print(f"Error with the speech recognition service: {e}")
            return None

In [6]:
import os
import csv
import datetime

def analyze_files_in_directory(directory_path, output_csv='file_analysis.csv'):
    """
    Analyze all files in a specified directory and save their details to a CSV file.
    
    Args:
        directory_path (str): Path to the directory to analyze
        output_csv (str): Path where the output CSV will be saved
    """
    try:
        with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
            # Define CSV headers
            fieldnames = [
                'file_path',
                'file_name',
                'file_extension',
                'parent_folder',
                'creation_time',
                'last_modified',
                'size_bytes',
                'depth_level'
            ]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

            # Walk through the directory
            for root, dirs, files in os.walk(directory_path):
                # Calculate depth level
                depth = root[len(directory_path):].count(os.sep)
                
                for file in files:
                    try:
                        full_file_path = os.path.join(root, file)
                        file_stat = os.stat(full_file_path)
                        
                        # Get file extension
                        _, file_extension = os.path.splitext(file)
                        
                        # Write file information to CSV
                        writer.writerow({
                            'file_path': full_file_path,
                            'file_name': file,
                            'file_extension': file_extension,
                            'parent_folder': os.path.basename(root),
                            'creation_time': datetime.datetime.fromtimestamp(file_stat.st_ctime).isoformat(),
                            'last_modified': datetime.datetime.fromtimestamp(file_stat.st_mtime).isoformat(),
                            'size_bytes': file_stat.st_size,
                            'depth_level': depth
                        })
                    except PermissionError:
                        print(f"Permission denied for file: {file}")
                        continue
                    except Exception as file_error:
                        print(f"Error processing file {file}: {file_error}")
                        continue
                        
        print(f"File analysis complete. Results saved to {output_csv}")
        
        # Generate summary statistics
        with open(output_csv, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            rows = list(reader)
            
            total_files = len(rows)
            total_size = sum(int(row['size_bytes']) for row in rows)
            extensions = {}
            for row in rows:
                ext = row['file_extension'].lower()
                extensions[ext] = extensions.get(ext, 0) + 1
                
            print("\nSummary Statistics:")
            print(f"Total files analyzed: {total_files}")
            print(f"Total size: {total_size / (1024*1024):.2f} MB")
            print("\nFile extensions found:")
            for ext, count in sorted(extensions.items(), key=lambda x: x[1], reverse=True):
                print(f"{ext or 'no extension'}: {count} files")
            
    except Exception as e:
        print(f"An error occurred during file analysis: {e}")

In [16]:
import os
import csv
import subprocess
from difflib import SequenceMatcher
from typing import Dict, List, Tuple

def word_to_number(word: str) -> int:
    """Convert word numbers to integers."""
    number_mapping = {
        'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5,
        'six': 6, 'seven': 7, 'eight': 8, 'nine': 9, 'ten': 10
    }
    return number_mapping.get(word.lower(), 0)

def get_similarity_ratio(str1: str, str2: str) -> float:
    """Calculate similarity ratio between two strings."""
    return SequenceMatcher(None, str1.lower(), str2.lower()).ratio()

def find_best_matching_folders(target_name: str, folder_list: List[Dict], threshold: float = 0.3, max_matches: int = 3) -> List[Tuple[float, Dict]]:
    """Find the best matching folders based on name similarity."""
    matches = []
    
    for folder in folder_list:
        folder_name = folder['folder_name']
        similarity = get_similarity_ratio(target_name, folder_name)
        
        if similarity >= threshold:
            matches.append((similarity, folder))
    
    matches.sort(key=lambda x: x[0], reverse=True)
    return matches[:max_matches]

def handle_folder_selection(matches: List[Tuple[float, Dict]]) -> Dict:
    """Handle folder selection through voice commands."""
    while True:
        folder_selected = listen()
        
        if "choose" in folder_selected:
            number_word = folder_selected.replace("choose", "").strip()
            
            choice = word_to_number(number_word)
            
            if 1 <= choice <= len(matches):
                return matches[choice - 1][1]
            else:
                print(f"Invalid selection. Please choose a number between 1 and {len(matches)}")
                print("Say 'choose' followed by the number (e.g., 'choose one')")
        else:
            print("Please say 'choose' followed by the number (e.g., 'choose one')")

def command_input():
    csv_path = os.path.join('user_folders_analysis.csv')
    command = listen()
    
    if "open folder" in command:
        file_name = command.replace("open folder", "").strip()
        
        try:
            with open(csv_path, 'r') as csvfile:
                csv_reader = csv.DictReader(csvfile)
                folder_list = list(csv_reader)
                
                if not folder_list:
                    print("No folders found in the analysis file.")
                    return
                
                matches = find_best_matching_folders(file_name, folder_list)
                
                if not matches:
                    print(f"No folders found matching '{file_name}'")
                    return
                
                if len(matches) == 1:
                    similarity, best_match = matches[0]
                    folder_path = best_match['folder_path']
                    print(f"Opening folder: {best_match['folder_name']} (similarity: {similarity:.2%})")
                    subprocess.call(["xdg-open", folder_path])
                    analyze_files_in_directory(folder_path)
                else:
                    print("\nMultiple matching folders found:")
                    for i, (similarity, folder) in enumerate(matches, 1):
                        print(f"{i}. {folder['folder_name']} (similarity: {similarity:.2%})")
                    
                    print("\nPlease say 'choose' followed by the number (e.g., 'choose one')")
                    
                    chosen_folder = handle_folder_selection(matches)
                    folder_path = chosen_folder['folder_path']
                    print(f"\nOpening folder: {chosen_folder['folder_name']}")
                    subprocess.call(["xdg-open", folder_path])
                    analyze_files_in_directory(folder_path)
                
        except FileNotFoundError:
            print("Folder analysis CSV not found. Run the folder analysis script first.")
        except Exception as e:
            print(f"An error occurred: {e}")
    else:
        print("Command not recognized. Please say 'open folder' followed by the folder name.")

In [17]:
command_input()

ALSA lib pcm_dsnoop.c:601:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm_dsnoop.c:601:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664

Listening...
User said:  open folder desk

Multiple matching folders found:
1. disk (similarity: 75.00%)
2. Desktop (similarity: 72.73%)
3. Videos (similarity: 60.00%)

Please say 'choose' followed by the number (e.g., 'choose one')
Listening...


ALSA lib pcm_dsnoop.c:601:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm_dsnoop.c:601:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664

User said:  choose two

Opening folder: Desktop


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/vaibhav/snap/code/common/.cache/gio-modules/libgiolibproxy.so


File analysis complete. Results saved to file_analysis.csv

Summary Statistics:
Total files analyzed: 2
Total size: 0.02 MB

File extensions found:
no extension: 2 files


In [ ]:
import os
import csv
import datetime

def analyze_user_created_folders(output_csv='user_folders_analysis.csv'):
    """
    Scan and analyze user-created folders in the home directory
    """
    # Get the current user's home directory
    home_dir = os.path.expanduser('~')
    
    try:
        with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
            # CSV headers
            fieldnames = [
                'folder_path', 
                'folder_name', 
                'parent_folder',
                'creation_time', 
                'last_modified',
                'num_of_files',
                'total_size_bytes'
            ]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            
            # Directories to skip (system, hidden directories, and node modules)
            skip_dirs = [
                '.cache', 
                '.config', 
                '.local', 
                '.thumbnails', 
                '.npm', 
                '.docker',
                'snap',
                'flatpak',
                'node_modules'
            ]
            
            # Walk through user's home directory
            for root, dirs, files in os.walk(home_dir):
                # Remove system, hidden directories, and node_modules from search
                dirs[:] = [d for d in dirs 
                           if not d.startswith('.') 
                           and d not in skip_dirs
                           and d != 'node_modules']
                
                for folder in dirs:
                    try:
                        full_folder_path = os.path.join(root, folder)
                        
                        # Skip node_modules explicitly
                        if 'node_modules' in full_folder_path:
                            continue
                        
                        # Get folder stats
                        folder_stat = os.stat(full_folder_path)
                        
                        # Count files and calculate total size
                        num_files = 0
                        total_size = 0
                        for dirpath, _, filenames in os.walk(full_folder_path):
                            # Additional check to skip node_modules during walk
                            if 'node_modules' in dirpath:
                                continue
                            
                            for f in filenames:
                                file_path = os.path.join(dirpath, f)
                                try:
                                    num_files += 1
                                    total_size += os.path.getsize(file_path)
                                except:
                                    pass
                        
                        # Write folder information to CSV
                        writer.writerow({
                            'folder_path': full_folder_path,
                            'folder_name': folder,
                            'parent_folder': os.path.basename(root),
                            'creation_time': datetime.datetime.fromtimestamp(folder_stat.st_ctime).isoformat(),
                            'last_modified': datetime.datetime.fromtimestamp(folder_stat.st_mtime).isoformat(),
                            'num_of_files': num_files,
                            'total_size_bytes': total_size
                        })
                    
                    except PermissionError:
                        # Skip folders we can't access
                        continue
                    except Exception as folder_error:
                        print(f"Error processing folder {folder}: {folder_error}")
        
        print(f"User folders analysis complete. File saved as {output_csv}")
    
    except Exception as e:
        print(f"An error occurred during folder analysis: {e}")

analyze_user_created_folders()

User folders analysis complete. File saved as user_folders_analysis.csv


In [10]:
def execute_command(command):
    try:
        if "open" in command:
            file_name = command.replace("open", "").strip()
            subprocess.call(["xdg-open", file_name])
        elif "install" in command:
            package = command.replace("install", "").strip()
            os.system(f"sudo apt install {package}")
        elif "monitor" in command:
            os.system("htop")
        else:
            speak("Command not recognized.")
    except Exception as e:
        speak(f"Error: {str(e)}")

In [ ]:
if __name__ == "__main__":
    speak("Hello! Say 'Hey Tommy' to activate me.")
    
    while True:
        print("Waiting for 'Hey Tommy'...")
        command = listen()
        
        if command and "hey tommy" in command:
            speak("I am listening. What would you like to say?")
            print("I am listening. What would you like to say?")
            
            # Listen for the user's input after the wake word
            user_input = listen()
            
            if user_input:
                print(f"You said: {user_input}")
                speak(f"You said: {user_input}")
            else:
                speak("I didn't catch that. Please try again.")